In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import markovify
import re

In [2]:
stopwords_rus = stopwords.words('russian')

In [3]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [4]:
df['text'].apply(lambda x: len(x.split())).mean()

12.26488092613982

In [5]:
df.groupby('label').count()

,text
label,
negative,111923
positive,114911


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.label)

In [7]:
%%time

pp = Pipeline([('vectorizer', TfidfVectorizer(ngram_range=(1, 2), tokenizer=TweetTokenizer().tokenize, stop_words=stopwords_rus)), 
               ('classifier', 
                LogisticRegression()
               )
              ])
pp.fit(X_train, y_train)

c:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 20.1 s


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['и', 'в', 'во', 'не', 'что', 'он',
                                             'на', 'я', 'с', 'со', 'как'...
                                 tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x0000012202F93188>>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual

In [8]:
print(classification_report(pp.predict(X_train), y_train))
print(classification_report(pp.predict(X_test), y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     83961
    positive       1.00      1.00      1.00     86164

    accuracy                           1.00    170125
   macro avg       1.00      1.00      1.00    170125
weighted avg       1.00      1.00      1.00    170125

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27862
    positive       1.00      1.00      1.00     28847

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [9]:
pp.predict_proba(['asdf', 'asdf'])

array([[0.53084827, 0.46915173],
       [0.53084827, 0.46915173]])

In [10]:
all_tweets = df['text'].apply(lambda x: ' '.join([i for i in TweetTokenizer().tokenize(x.lower()) 
                                                  if ('http' not in i) and ('@' not in i) and (i not in ['rt', ':'])]))
all_tweets = '\n'.join(list(all_tweets))

In [11]:
all_model = markovify.NewlineText(all_tweets)

In [110]:
%%time

generated_all = []
for i in range(10000):
    generated_all.append(all_model.make_sentence())

Wall time: 37.1 s


In [13]:
positive_tweets = df[df['label'] == 'positive']['text'].apply(lambda x: ' '.join([i for i in TweetTokenizer().tokenize(x.lower()) 
                                                  if ('http' not in i) and ('@' not in i) and (i not in ['rt', ':'])]))
positive_tweets = '\n'.join(list(positive_tweets))

In [14]:
len(all_tweets)

16473696

In [15]:
positive_model = markovify.NewlineText(positive_tweets)

In [111]:
%%time

generated_positive = []
for i in range(10000):
    generated_positive.append(positive_model.make_sentence())

Wall time: 29.5 s


In [17]:
negative_tweets = df[df['label'] == 'negative']['text'].apply(lambda x: ' '.join([i for i in TweetTokenizer().tokenize(x.lower()) 
                                                  if ('http' not in i) and ('@' not in i) and (i not in ['rt', ':'])]))
negative_tweets = '\n'.join(list(negative_tweets))

In [18]:
negative_model = markovify.NewlineText(negative_tweets)

In [112]:
%%time

generated_negative = []
for i in range(10000):
    generated_negative.append(negative_model.make_sentence())

Wall time: 10.1 s


In [47]:
positive_model.make_sentence()

'да все норм :d ибо на столе , если через пять минут назад . до свидания , мир .'

In [107]:
def punct_fix(text):
    if re.search(r' [.,!?]+?', text):
        text = re.sub(r' ([.,!?])+?', r'\1', text)
        
    return text

In [113]:
for i in [generated_all, generated_positive, generated_negative]:
    for n, j in enumerate(i):
        if j:
            i[n] = punct_fix(j)
        else:
            i.remove(j)

In [114]:
generated_positive

['не плохо полы помыть, но счастлив :d',
 'кирилл ауди открыл, его кто-то смотрит? ^ _ ^ /.. / #teamfollowback',
 ':d дочь в меня влюбился. самому не вериться :d',
 'засну как вариант :d d',
 'чем украсить ёлку? — ой да не, ну как хочешь ты :d оставайся такой, интересно',
 'че, смотреть на то, что он гей :d',
 'в тэшке меня задавила жируха, день прошел? :d субботу и воскресение',
 'пришла домой, а у меня красивый перебинтованный пальчик :d 1983 год',
 'зато я 2 дня погулять, но с ней ^ ^ жду нашей встречи :d',
 'он уже с тобой все ближе и ближе. эта запись наберет 1000 лайков, забаню всех фейков, срущих в комментах :d',
 'пока микатса делала вавилонскую стену, когда спишь с плюшевым мишкой, он прям аащащщащаща :d dd',
 'ну... соблазнительная она или нет :d теперь вспомнила :d',
 'глаза красные, визина нет, нет меня не только утро',
 'ещё один cm / изи бризи если в вашей жизни есть я :d',
 'последнее время жители курортов турции думают, что типа это рашн нейм лалалал, и чтобы уже состоя

In [106]:
for i in [generated_all, generated_positive, generated_negative]:
    for j in i:
        if not j:
            print(j)

None
None
None
None
None
